In [2]:
from pyspark.sql.types import *
%load_ext autoreload
%autoreload 2

import pyspark, os, sys
from delta import *
from dags.lib.IncrementalLoader import IncrementalLoader
from dags.lib.Processer import *
from pyspark.sql.functions import col, lit, max as spark_max, length, row_number, explode, unix_timestamp, from_unixtime

ModuleNotFoundError: No module named 'lib'

In [3]:
def create_spark_session():
    conf = (
        pyspark.conf.SparkConf()
        .setAppName("LetsTalk")
        .set(
            "spark.sql.catalog.spark_catalog",
            "org.apache.spark.sql.delta.catalog.DeltaCatalog",
        )
        .set("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
        .set("spark.hadoop.fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
        .set("spark.hadoop.google.cloud.auth.service.account.enable", "true")
        .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", "/Users/alfio/projects/upc/BDMP2/docker/gcs.json")
        .set("spark.sql.shuffle.partitions", "4")
        .set("spark.jars", "../docker/gcs-connector-hadoop.jar")
        .setMaster(
            "local[*]"
        )
    )

    builder = pyspark.sql.SparkSession.builder.appName("LetsTalk").config(conf=conf)
    spark = configure_spark_with_delta_pip(builder).getOrCreate()
    return spark

In [5]:
spark = create_spark_session()
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)
absolute_path_to_landing = '/Users/alfio/projects/upc/BDMP2/data/letstalk_landing_zone_bdma'
absolute_path_to_trusted = '/Users/alfio/projects/upc/BDMP2/data/letstalk_trusted_zone_bdma'


:: loading settings :: url = jar:file:/Users/alfio/python_venv/general/lib/python3.13/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/alfio/.ivy2/cache
The jars for the packages stored in: /Users/alfio/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-6bc098b8-7ec2-4ec2-8d4a-6021c5e25fd0;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.3.0 in central
	found io.delta#delta-storage;3.3.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 73ms :: artifacts dl 3ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.3.0 from central in [default]
	io.delta#delta-storage;3.3.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0  

In [8]:
table_subpath = f'delta_tmdb/now_playing'

loader = IncrementalLoader(spark, absolute_path_to_landing, table_subpath)
df = loader.get_new_data()


25/05/30 18:21:52 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


INFO:dags.lib.pt_utils:CDF not available — doing full load


In [27]:
processor= TMDBProcessor(spark, df)

processor.ensure_schema()
processor.normalize_text(['overview', 'title', 'original_title'])
processor.ensure_schema_movie_genres()
table_subpath = "delta_tmdb\\now_playing"
processor.type_dump(absolute_path_to_trusted, table_subpath)


/Users/alfio/projects/upc/BDMP2/data/letstalk_trusted_zone_bdma/now_playing
+-------+
|film_id|
+-------+
| 324544|
| 950387|
|1197306|
|1045938|
|1195506|
|1293286|
| 822119|
|1125899|
|1229730|
|1244944|
|1353117|
| 696506|
|1373723|
|1261050|
|1282980|
|1087891|
|1212855|
|1126166|
|1297763|
| 447273|
+-------+
only showing top 20 rows



In [8]:
df = spark.read.format("delta").load("gs://letstalk_landing_zone_bdma/control_table")

In [7]:
df = df.filter(~df.source_table.startswith("delta_tmdb"))
df.write.format("delta").mode("overwrite").save("gs://letstalk_landing_zone_bdma/control_table")

In [9]:
df

source_table,last_processed_version,last_run_ts
delta_news/entert...,42,2025-05-30 17:33:...
delta_news/techno...,42,2025-05-30 17:34:...
delta_sports/matches,61,2025-05-29 19:16:...
delta_sports/leagues,86,2025-05-29 19:16:...
delta_news/sports,42,2025-05-30 17:34:...


In [17]:
processor.df
processor.movie_genre_df

[Row(teams=Row(away=Row(id=18814, logo='https://media.api-sports.io/football/teams/18814.png', name='Minnesota United II', winner=True), home=Row(id=4102, logo='https://media.api-sports.io/football/teams/4102.png', name='Colorado Rapids II', winner=False)))]

In [20]:
sproc.teams.groupBy('team_id').count().filter('count>1')
sproc.venues.groupBy('venue_id').count().filter('count>1')


venue_id,count


In [22]:
sproc.merge_with_trusted("../data/letstalk_trusted_zone_bdma", 'delta_sports', ['fixture_id', 'status_long'])
loader.update_control_table()

INFO:dags.lib.pt_utils:Saving unique records from overlapping ones


--- Logging error ---
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/python@3.13/3.13.3/Frameworks/Python.framework/Versions/3.13/lib/python3.13/logging/__init__.py", line 1150, in emit
    msg = self.format(record)
  File "/opt/homebrew/Cellar/python@3.13/3.13.3/Frameworks/Python.framework/Versions/3.13/lib/python3.13/logging/__init__.py", line 998, in format
    return fmt.format(record)
           ~~~~~~~~~~^^^^^^^^
  File "/opt/homebrew/Cellar/python@3.13/3.13.3/Frameworks/Python.framework/Versions/3.13/lib/python3.13/logging/__init__.py", line 711, in format
    record.message = record.getMessage()
                     ~~~~~~~~~~~~~~~~~^^
  File "/opt/homebrew/Cellar/python@3.13/3.13.3/Frameworks/Python.framework/Versions/3.13/lib/python3.13/logging/__init__.py", line 400, in getMessage
    msg = msg % self.args
          ~~~~^~~~~~~~~~~
TypeError: not all arguments converted during string formatting
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as

INFO:dags.lib.pt_utils:Added new 0 unique records to table at path ../data/letstalk_trusted_zone_bdma/delta_sports/teams
INFO:dags.lib.pt_utils:Saving unique records from overlapping ones


--- Logging error ---
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/python@3.13/3.13.3/Frameworks/Python.framework/Versions/3.13/lib/python3.13/logging/__init__.py", line 1150, in emit
    msg = self.format(record)
  File "/opt/homebrew/Cellar/python@3.13/3.13.3/Frameworks/Python.framework/Versions/3.13/lib/python3.13/logging/__init__.py", line 998, in format
    return fmt.format(record)
           ~~~~~~~~~~^^^^^^^^
  File "/opt/homebrew/Cellar/python@3.13/3.13.3/Frameworks/Python.framework/Versions/3.13/lib/python3.13/logging/__init__.py", line 711, in format
    record.message = record.getMessage()
                     ~~~~~~~~~~~~~~~~~^^
  File "/opt/homebrew/Cellar/python@3.13/3.13.3/Frameworks/Python.framework/Versions/3.13/lib/python3.13/logging/__init__.py", line 400, in getMessage
    msg = msg % self.args
          ~~~~^~~~~~~~~~~
TypeError: not all arguments converted during string formatting
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as

INFO:dags.lib.pt_utils:Added new 0 unique records to table at path ../data/letstalk_trusted_zone_bdma/delta_sports/venues


--- Logging error ---
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/python@3.13/3.13.3/Frameworks/Python.framework/Versions/3.13/lib/python3.13/logging/__init__.py", line 1150, in emit
    msg = self.format(record)
  File "/opt/homebrew/Cellar/python@3.13/3.13.3/Frameworks/Python.framework/Versions/3.13/lib/python3.13/logging/__init__.py", line 998, in format
    return fmt.format(record)
           ~~~~~~~~~~^^^^^^^^
  File "/opt/homebrew/Cellar/python@3.13/3.13.3/Frameworks/Python.framework/Versions/3.13/lib/python3.13/logging/__init__.py", line 711, in format
    record.message = record.getMessage()
                     ~~~~~~~~~~~~~~~~~^^
  File "/opt/homebrew/Cellar/python@3.13/3.13.3/Frameworks/Python.framework/Versions/3.13/lib/python3.13/logging/__init__.py", line 400, in getMessage
    msg = msg % self.args
          ~~~~^~~~~~~~~~~
TypeError: not all arguments converted during string formatting
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as

INFO:dags.lib.pt_utils:Saving unique records from overlapping ones
INFO:dags.lib.pt_utils:Added new 0 unique records
INFO:dags.lib.pt_utils:Appending non overlapping records
INFO:dags.lib.pt_utils:Adding new 0 records


In [ ]:
full_distinct_df = sproc.df.distinct()
print(full_distinct_df.count())

partial_distinct_df = sproc.df.select('fixture_id', 'status_long').distinct()
print(partial_distinct_df.count())


In [ ]:
sproc.merge_with_trusted("../data/letstalk_trusted_zone_bdma", 'delta_sports', ['fixture_id', 'status_long'])


In [ ]:
sproc.merge_with_trusted("../data/letstalk_trusted_zone_bdma", 'delta_sports', ['fixture_id', 'status_long'])
